In [1]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("full_dataset.csv")
data = data.sample(frac=1)
data.head()

,Questions,Answers,Category,Sentiment
141162,percentag undergrad live notr dame campu,80%,squad,4.0
11993,call tree hate joke,A face palm.,humor,4.0
32782,wayn gretzki wife call,"Wayne""",humor,5.0
102039,mani fono member,49,squad,2.0
21161,what blue yellow sit bottom pool,A baby with slashed floaties. What's red and ...,humor,4.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142799 entries, 77156 to 95282
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Questions  142799 non-null  object 
 1   Answers    142799 non-null  object 
 2   Category   142799 non-null  object 
 3   Sentiment  142799 non-null  float64
dtypes: float64(1), object(3)
memory usage: 5.4+ MB


In [4]:
answer_encoder = LabelEncoder()
answer_encoder.fit(data['Answers'])
answer_labels = answer_encoder.transform(data['Answers'])

In [5]:
sentiment_encoder = LabelEncoder()
sentiment_encoder.fit(data['Sentiment'])
sentiment_labels = sentiment_encoder.transform(data['Sentiment'])

In [6]:
vocab_size = 30000
embedding_dim = 300
max_len = 128
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(data['Questions'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(data['Questions'])
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
del word_index
del sequences

In [7]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_len)(input_layer)
global_average_layer = GlobalAveragePooling1D()(embedding_layer)
first_dense = Dense(units=256, activation='relu')(global_average_layer)
dropout_1 = Dropout(0.2)(first_dense)
first_output = Dense(units=len(data['Questions']), name='answer_output', activation='softmax')(dropout_1)
second_dense = Dense(units=256, activation='relu')(first_dense)
dropout_2 = Dropout(0.2)(second_dense)
second_output = Dense(units=5, name='sentiment_output', activation='softmax')(dropout_2)
model = Model(inputs=input_layer, outputs=[first_output, second_output])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 300)     9000000     ['input_1[0][0]']                
                                                                                                  
 global_average_pooling1d (Glob  (None, 300)         0           ['embedding[0][0]']              
 alAveragePooling1D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 256)          77056       ['global_average_pooling1d[0]

In [9]:
callback_ES = EarlyStopping(monitor='loss', patience=10)

In [10]:
epochs = 150
history = model.fit(padded_sequences, [np.array(answer_labels), np.array(sentiment_labels)], epochs = epochs, callbacks=[callback_ES], batch_size=100)

model.save("current_model")
import pickle

with open('current_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('current_answer_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(answer_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('current_sentiment_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(sentiment_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Epoch 1/150
1428/1428 [==============================] - 225s 155ms/step - loss: 12.9285 - answer_output_loss: 11.5781 - sentiment_output_loss: 1.3504 - answer_output_accuracy: 0.0027 - sentiment_output_accuracy: 0.3508
Epoch 2/150
1428/1428 [==============================] - 222s 155ms/step - loss: 12.0015 - answer_output_loss: 10.9804 - sentiment_output_loss: 1.0211 - answer_output_accuracy: 0.0047 - sentiment_output_accuracy: 0.5480
Epoch 3/150
1428/1428 [==============================] - 221s 155ms/step - loss: 11.3443 - answer_output_loss: 10.4711 - sentiment_output_loss: 0.8731 - answer_output_accuracy: 0.0085 - sentiment_output_accuracy: 0.6278
Epoch 4/150
1428/1428 [==============================] - 222s 156ms/step - loss: 10.6868 - answer_output_loss: 9.8769 - sentiment_output_loss: 0.8099 - answer_output_accuracy: 0.0142 - sentiment_output_accuracy: 0.6546
Epoch 5/150
1428/1428 [==============================] - 222s 155ms/step - loss: 10.0880 - answer_output_loss: 9.3162 - s

In [ ]:
plt.plot(history.history['loss'])
plt.title('loss value')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()